In [1]:
# necessary imports
import numpy as np
import os
import seaborn as sns
import scipy.stats as stat
from scipy.stats import pearsonr
from scipy.stats import pointbiserialr
import pandas as pd
import matplotlib.pyplot as plt
from pydataset import data
import warnings
warnings.filterwarnings("ignore")

import wrangle as wra

#testing

In [2]:
# build query and look at data
zillow_query = """
        select *
        from properties_2017
        left join predictions_2017 using (parcelid)
        left join airconditioningtype using (airconditioningtypeid)
        left join architecturalstyletype using (architecturalstyletypeid)
        left join buildingclasstype using (buildingclasstypeid)
        left join heatingorsystemtype using (heatingorsystemtypeid)
        left join propertylandusetype using (propertylandusetypeid)
        left join storytype using (storytypeid)
        left join typeconstructiontype using (typeconstructiontypeid)
            where YEAR(transactiondate) = 2017;
        """

df = wra.get_zillow_data(zillow_query)

df.head()

,Unnamed: 0,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,...,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,0,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14297519,1727539,...,0,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17052889,1387261,...,1,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,2,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14186244,11677,...,2,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,12177905,2288172,...,3,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,4,NaN,NaN,266.0,2.0,NaN,NaN,1.0,10887214,1970746,...,4,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [3]:
wra.wrangle_zillow(df)

,propertylandusetypeid,heatingorsystemtypeid,parcelid,id,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id.1,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
0,261.0,24.0,14297519,1727539,3.5,4.0,3.5,3100.0,3100.0,6059.0,...,1023282.0,2016.0,537569.0,11013.72,6.059063e+13,0,0.025595,2017-01-01,Yes,Single Family Residential
1,261.0,24.0,17052889,1387261,1.0,2.0,1.0,1465.0,1465.0,6111.0,...,464000.0,2016.0,376000.0,5672.48,6.111001e+13,1,0.055619,2017-01-01,Yes,Single Family Residential
2,261.0,24.0,14186244,11677,2.0,3.0,2.0,1243.0,1243.0,6059.0,...,564778.0,2016.0,479489.0,6488.30,6.059022e+13,2,0.005383,2017-01-01,Yes,Single Family Residential
3,261.0,2.0,12177905,2288172,3.0,4.0,3.0,2376.0,2376.0,6037.0,...,145143.0,2016.0,36225.0,1777.51,6.037300e+13,3,-0.103410,2017-01-01,Central,Single Family Residential
4,266.0,2.0,10887214,1970746,3.0,3.0,3.0,1312.0,1312.0,6037.0,...,119407.0,2016.0,45726.0,1533.89,6.037124e+13,4,0.006940,2017-01-01,Central,Condominium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77608,266.0,2.0,10833991,2864704,3.0,3.0,3.0,1741.0,1741.0,6037.0,...,379000.0,2016.0,114000.0,4685.34,6.037132e+13,77608,-0.002245,2017-09-20,Central,Condominium
77609,261.0,2.0,11000655,673515,2.0,2.0,2.0,1286.0,1286.0,6037.0,...,354621.0,2016.0,283704.0,4478.43,6.037101e+13,77609,0.020615,2017-09-20,Central,Single Family Residential
77610,261.0,24.0,17239384,2968375,2.0,4.0,2.0,1612.0,1612.0,6111.0,...,67205.0,2016.0,16522.0,1107.48,6.111008e+13,77610,0.013209,2017-09-21,Yes,Single Family Residential
77611,261.0,2.0,12773139,1843709,1.0,3.0,1.0,1032.0,1032.0,6037.0,...,49546.0,2016.0,16749.0,876.43,6.037434e+13,77611,0.037129,2017-09-21,Central,Single Family Residential


### Ask at least 5 questions about the data, keeping in mind that your target variable is logerror. e.g. Is logerror significantly different for properties in LA County vs Orange County vs Ventura County?

### Answer those questions through a mix of statistical tests and visualizations.